# Linear models  

In this notebook we will be looking at ways to use linear models to predict electricity demand for the GTA. We will trying different combinations of features through best subset selection which in the end should give us a sense of the most important features.

In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection
import sklearn.linear_model
import math

In [ ]:
# Load the data

In [ ]:
# Create a new column 't' which is just the index of each date starting at 0

To account for the seasonality in the data it will not be enough to consider a linear model of the form $X_t = \beta_1 t + \beta_0$ as this would only pickup on the underlying linear trend in the data (for which we already suspect is relatively weak according to our intial plots and time series decomposition <b>Note to self: this may be subject to change depending on how the time series analysis goes</b>).

We need to introduce some seasonal behaviour in our linear model. One way to do this (<b>CITE HERE</b>) is through fourier feature bases. If $m$ is the seasonal period we suspect (e.g. daily, weekly, monthly, yearly) then we can introduce terms of the following form:
$X_t = \sum_{j=1}^P \beta_{2j-1} \sin(\frac{2 \pi j t}{m}) + \beta_{2j} \cos(\frac{2 \pi j t}{m})$
where $m$ is the the seasonal period and $P$ is the number of pairs of fourier series we have. We will bound $P$ to at most $\frac{m}{2}$. For daily seasonality $m=24$, for weekly $m=7 \times 24$, for monthly $m = 4 \times 7 \times 24$ (using the running assumption that we use a month as 4 weeks) and yearly as $365 \times 24 = 8760$.


In [ ]:
# Add columns for each of the fourier bases from j = 1 to P and for each different seasonality
# for j = 1 to m/2, but do this in ranges:
# for m = 24 use the range [1,...12]
# For m = 7*24 use a longer range like [1, 6, 12, ..., P] (so we don't blow up training)
# For m = 4*7*42 use a longer range [1, 12, ... P] (remember P = m/2 use P - 1 to be safe actually)
# Again the same thing for m = 8760 use a larger ranger something divisible by 8760 like 10, 20

In [ ]:
# For each possible pair train the model with cross validation with the following function
#The model is the sklearn model, the n_splits is the number of folds (try to find a reasonable
# size but i suspect we won't be able to do it a large number of splits, there's just
# too much data! )
# X is the transformed set of features and y is the target electricity demand
def rolling_window_time_series(model, n_splits, X, y):
    # Keep a running total of the mean mse
    mse_cv = 0
    N = 1
    ts_splitter = sklearn.model_selection.TimeSeriesSplit(n_splits=n_splits)
    for train, valid in ts_splitter.split(X):
        # For each train, valid fold fit it on train
        model.fit(X[train], y[train])
        # Predict on valid
        y_pred = model.predict(X[valid])
        # Find mse of this fold and then add it to the running average
        mse_fold = sklearn.metrics.mean_squared_error(y_true=y[valid], y_pred=y_pred)
        # This is to keep a running average (instead of appending to an array and then
        # taking the mean)
        if N == 1:
            mse_cv = mse_fold
        else:
            # This can be checked by expanding the formula for the mean
            mse_cv += (mse_fold - mse_cv)/N
        N+=1
    # Return mse and rmse
    return mse_cv, math.sqrt(mse_cv)

# 1. So high level it's like a for loop over all possible combiantions of parameters
# 2. For each of those combinations train the model using cross validation
# Record the average mse => 2. should be done by the function rolling_window

In [1]:
# Once you get the optimal fourier series linear_model
# Do best subset selection on the 10 features so 2^10 combinations now added to the 
# best model you found above